In [2]:
!pip install pyspark

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [5]:
from csv import reader
from pyspark.sql import Row 
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import warnings
import pyspark.sql.functions as F
from pyspark.sql.functions import udf, col, isnan, when, count, trim, desc, sum, asc
from pyspark.sql.functions import countDistinct, explode, split, concat_ws, collect_list
from pyspark.sql.functions import lit
from pyspark.sql.types import StructType as R, StructField as Fld, DoubleType as Dbl, StringType as Str, IntegerType as Int, DateType as Date

import os
os.environ["PYSPARK_PYTHON"] = "python3"

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [6]:
spark = SparkSession \
    .builder \
    .appName("antioxidants analysis") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [7]:
dSchema = R([
            Fld("Product",Str()),
            Fld("origin",Str()),
            Fld("Procured_in",Str()),
            Fld("antioxidant_content_mmol_100g", Dbl())
            ])

In [10]:
#read in tables
dffruits = spark.read.csv("../input/fruits/Fruit.csv", header=True, schema=dSchema)
dfvegetables = spark.read.csv("../input/vegetables/Vegetables.csv", header = True, schema=dSchema)
dfnuts = spark.read.csv("../input/nuts-and-seeds/Nuts and seeds.csv", header = True, schema=dSchema)

In [ ]:
dffruits = dffruits.withColumn('category', lit('Fruit'))
dffruits.show(5, truncate = True)
dffruits.count()